In [1]:
library(purrr)
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
"running command 'timedatectl' had status 1"
-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.5     v dplyr   1.0.7
v tibble  3.1.6     v stringr 1.4.0
v tidyr   1.1.4     v forcats 0.5.1
v readr   2.1.0     

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
source('paths.R')
source('misc.R')
source('factors.R')

In [3]:
m <- load_object(path_at('out', 'cache')('annotated', 'metadata', 'Robj'))

In [4]:
subcluster_long <- function(cell_type_) (
    m
    %>% filter(cell_type == cell_type_)
    %>% restore_subcluster()
    %>% filter(!is.na(subcluster))
    %>% select(condition_short, subcluster)    
    %>% rename(category=subcluster)
)
    
cell_type_long <- function() (
    m
    %>% filter(!is.na(cell_type))
    %>% select(condition_short, cell_type)
    %>% rename(category=cell_type)
)

ct.split <- list(cell_type=cell_type_long())
subc.splits <- (cell_type_levels %>% set_names() %>% map(subcluster_long))
splits <- append(ct.split, subc.splits)

In [5]:
tabulate_ <- function(long) (
    long
    %>% group_by(category, condition_short)
    %>% count()
    %>% ungroup()
    %>% pivot_wider(names_from=category, values_from=n)
    %>% arrange(condition_short)
    %>% mutate_if(is.numeric, ~replace_na(.x, 0))
)
tabs <- splits %>% map(tabulate_)
tabs[[5]]

condition_short,Steady state,Immature,Inflamed IFN-λ−,Inflamed IFN-λ+
<ord>,<dbl>,<dbl>,<dbl>,<dbl>
M,222,155,8,0
V,95,82,131,114
VCy,393,111,78,18
VIf,194,165,73,16
VCyIf,265,153,66,1
MCyIf,237,157,36,1


In [6]:
.chisq_test_matrix <- function(m) {
    if (1 == ncol(m) | 1 == nrow(m)) return(NA)
    r <- chisq.test(m, simulate.p.value = TRUE, B=1000000)
    #r <- chisq.test(m)
    return(r$p.value)
}

chisq_test <- function(data) (
    data
    %>% select_if(is.numeric)
    %>% select(where(~ any(. != 0)))
    %>% as.matrix()
    %>% .chisq_test_matrix()
)


In [7]:
condition_pairs_selectors <- (
    c('V_vs_M', 'VCy_vs_V', 'VIf_vs_V', 'VCyIf_vs_V', 'VCy_vs_VIf',
      'VCyIf_vs_VCy', 'VCyIf_vs_VIf', 'MCyIf_vs_M', 'VCyIf_vs_M')
    %>% set_names()
    %>% map(~str_split(.x, '_vs_')[[1]])
    %>% map(~function(df)(df %>% filter(condition_short %in% .x)))
)

test_condition_pairs <- function(tbl)(
    condition_pairs_selectors
    %>% map(~.x(tbl))
    %>% map(chisq_test)
)
    
p_vals <- tabs %>% map(test_condition_pairs)

In [8]:
(
    p_vals
    %>% bind_rows(.id = 'contrast') 
    %>% pivot_longer(where(is.numeric), names_to = 'condition_pair')
    %>% mutate(value = scales::pvalue(value, accuracy=1e-3))
    %>% pivot_wider(names_from=contrast, values_from=value)
)

condition_pair,cell_type,Basal,BdiS,Secretory,Ciliated
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
V_vs_M,<0.001,<0.001,<0.001,<0.001,<0.001
VCy_vs_V,<0.001,<0.001,<0.001,<0.001,<0.001
VIf_vs_V,<0.001,<0.001,<0.001,<0.001,<0.001
VCyIf_vs_V,0.002,<0.001,<0.001,<0.001,<0.001
VCy_vs_VIf,0.012,0.606,<0.001,<0.001,<0.001
VCyIf_vs_VCy,<0.001,<0.001,<0.001,<0.001,<0.001
VCyIf_vs_VIf,<0.001,<0.001,0.002,<0.001,<0.001
MCyIf_vs_M,0.011,<0.001,0.001,0.150,<0.001
VCyIf_vs_M,<0.001,<0.001,0.064,<0.001,<0.001
